## Assignment 4 Solution

### Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

### Data Pre-Processing

In [3]:
df = pd.read_csv('spam.csv', delimiter=',', encoding='latin-1')
df.head()

v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
X = df.v2
Y = df.v1
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
tokenizer = Tokenizer(num_words=2000, lower=True)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen=200)

### Create model

In [8]:
model = Sequential()

### Add layers

In [9]:
model.add(Embedding(2000, 50, input_length=200))
model.add(LSTM(64))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
_________________________________________________________________


### Compile the model

In [11]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

### Fit the model

In [12]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 4s 49ms/step - loss: 0.3426 - accuracy: 0.8738 - val_loss: 0.1774 - val_accuracy: 0.9585
Epoch 2/10
28/28 [==============================] - 1s 27ms/step - loss: 0.1026 - accuracy: 0.9745 - val_loss: 0.0607 - val_accuracy: 0.9809
Epoch 3/10
28/28 [==============================] - 1s 27ms/step - loss: 0.0417 - accuracy: 0.9882 - val_loss: 0.0606 - val_accuracy: 0.9832
Epoch 4/10
28/28 [==============================] - 1s 27ms/step - loss: 0.0253 - accuracy: 0.9927 - val_loss: 0.0579 - val_accuracy: 0.9843
Epoch 5/10
28/28 [==============================] - 1s 26ms/step - loss: 0.0191 - accuracy: 0.9947 - val_loss: 0.0744 - val_accuracy: 0.9865
Epoch 6/10
28/28 [==============================] - 1s 28ms/step - loss: 0.0131 - accuracy: 0.9961 - val_loss: 0.0762 - val_accuracy: 0.9865
Epoch 7/10
28/28 [==============================] - 1s 26ms/step - loss: 0.0085 - accuracy: 0.9969 - val_loss: 0.1080 - val_accuracy: 0.9854
Epoch 8/10
28

### Save the model

In [13]:
model.save("model.h5")

### Test the model

In [14]:
test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences, maxlen=200)

In [15]:
acc = model.evaluate(X_test, y_test)

35/35 [==============================] - 0s 12ms/step - loss: 0.0825 - accuracy: 0.9839


In [16]:
def predict(message):
    txt = tokenizer.texts_to_sequences(message)
    txt = sequence.pad_sequences(txt, maxlen=200)
    preds = model.predict(txt)
    if preds > 0.5:
        print("Spam")
    else:
        print("Not Spam")

In [19]:
predict(["Sorry, I'll call after the meeting."])

1/1 [==============================] - 0s 28ms/step
Not Spam


In [20]:
predict(["Congratulations!!! You won $50,000. Send message LUCKY100 to XXXXXXXXXX to recieve your prize."])

1/1 [==============================] - 0s 25ms/step
Spam
